In [1]:
#from dask.distributed import Client
import numpy as np
from scipy.interpolate import griddata  
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import intake
import dask
import multiprocessing
import xarray as xr
import sysd
ncpu = multiprocessing.cpu_count()
threads = 6
nworker = ncpu//threads
print(f'Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}')

Number of CPUs: 256, number of threads: 6, number of workers: 42


In [ ]:
client = Client(processes=True, threads_per_worker=threads, n_workers=nworker, memory_limit='32GB')
client

In [4]:
%ls /work/ik1017/Catalogs/

archive/                                 dkrz_data-pool_diskcatalog.yaml*
Candidates/                              dkrz_dyamond-winter_disk.json*
cmip6-retracted-outdated.csv.gz*         dkrz_era5_disk.csv.gz*
cmip6-retracted-outdated.json*           dkrz_era5_disk.json*
dkrz_catalog.yaml*                       dkrz_monsoon_disk.yaml
dkrz_cmip5_archive.csv.gz                dkrz_mpige_disk.csv.gz
dkrz_cmip5_archive.json*                 dkrz_mpige_disk.json*
dkrz_cmip5_disk.csv.gz*                  dkrz_nextgems_disk.json*
dkrz_cmip5_disk.json*                    dkrz_palmod2_disk.csv.gz*
dkrz_cmip5_disk_netcdf.csv.gz*           dkrz_palmod2_disk.json*
dkrz_cmip5_disk_netcdf.json*             highresmonsoon/
dkrz_cmip6_archive.csv.gz*               intake-esm_template_cmip6.json*
dkrz_cmip6_cloud.csv.gz                  mistral-cmip6-cmip.csv.gz*
dkrz_cmip6_cloud.json*                   mistral-cmip6-cmip.json*
dkrz_cmip6_disk.csv                      mistral-cmip6-pids.csv.gz*
dkrz_cm

In [3]:
%ls /pool/data/Catalogs/

Candidates/               dkrz_dyamond-winter_disk.json@
dkrz_catalog.yaml@        dkrz_era5_disk.json@
dkrz_cmip5_archive.json@  dkrz_monsoon_disk.yaml@
dkrz_cmip5_disk.json@     dkrz_mpige_disk.json@
dkrz_cmip6_cloud.json@    dkrz_nextgems_disk.json@
dkrz_cmip6_disk.json@     dkrz_palmod2_disk.json@
dkrz_cordex_disk.json@    Templates/


In [2]:
root_path = '/work/ik1017/Catalogs/'
#root_path = '/pool/data/Catalogs/Candidates/'
#root_path = '/pool/data/Catalogs/'
#col_url = f"{root_path}mistral-cmip6.json"
col_url = f"{root_path}dkrz_cmip-data-pool_disk_netcdf.json"
col_url = f"{root_path}dkrz_cmip6_disk.json"
col = intake.open_esm_datastore(col_url)
col

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/utils.py:96: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(catalog_path, **csv_kwargs), catalog_path


,unique
activity_id,20
institution_id,39
source_id,103
experiment_id,214
member_id,479
table_id,39
variable_id,1164
grid_label,12
dcpp_init_year,63
version,753


In [10]:
cloudpath="https://swift.dkrz.de/v1/dkrz_a44962e3ba914c309a7421573a6949a6/intake-esm/dkrz_data-pool_cloudcatalog.yaml"
cloudpath="https://gitlab.dkrz.de/data-infrastructure-services/intake-esm/-/raw/master/esm-collections/cloud-access/dkrz_catalog.yaml"
#poolpath="/pool/data/Catalogs/dkrz_cmip6_disk_netcdf.json"
cdp = intake.open_catalog(cloudpath)
col = cdp.dkrz_cmip6_disk#dkrz_cmip6_disk_netcdf_fromcloud
col.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,format,uri
0,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,c2h6,gn,NaN,v20200511,185001-201412,netcdf,/mnt/lustre/work/ik1017/CMIP6/data/CMIP6/AerCh...
1,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,c3h6,gn,NaN,v20200511,185001-201412,netcdf,/mnt/lustre/work/ik1017/CMIP6/data/CMIP6/AerCh...
2,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,c3h8,gn,NaN,v20200511,185001-201412,netcdf,/mnt/lustre/work/ik1017/CMIP6/data/CMIP6/AerCh...
3,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,cdnc,gn,NaN,v20200522,185001-201412,netcdf,/mnt/lustre/work/ik1017/CMIP6/data/CMIP6/AerCh...
4,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,ch3coch3,gn,NaN,v20200511,185001-201412,netcdf,/mnt/lustre/work/ik1017/CMIP6/data/CMIP6/AerCh...


In [9]:
print(list(cdp))


['dkrz_cmip5_disk', 'dkrz_cmip5_archive', 'dkrz_cmip6_disk', 'dkrz_cmip6_cloud', 'dkrz_cordex_disk', 'dkrz_era5_disk', 'dkrz_mpi-ge_disk', 'dkrz_dyamond-winter_disk']


In [40]:
col_url = "/home/b/b381258/swift.dkrz.de/dkrz_cmip6_disk_netcdf.csv" #dkrz_data-pool_cloudcatalog.yaml"
#col_url = f"{root_path}dkrz_data-pool_diskcatalog.yaml"
#parent_col=intake.open_catalog(col_url)
parent_col = intake.open_csv(col_url)
#print(list(parent_col))
#col=parent_col["dkrz_cmip6_swift_zarr_fromdisk"]
#cat = parent_col.search(**query)

In [41]:
parent_col.container

'dataframe'

In [5]:
models = ['CanESM5', 'MPI-ESM1-2-LR','CNRM-CM6-1']#'CESM2-WACCM']
query = dict(experiment_id='historical', table_id='day', 
             variable_id='ua', source_id=models)
cat = col.search(**query)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,...,frequency,time_reduction,long_name,units,realm,level_type,time_min,time_max,format,uri
0,CMIP,CCCma,CanESM5,historical,r10i1p1f1,day,ua,gn,NaN,v20190429,...,day,mean,Eastward Wind,m s-1,atmos,NaN,18500101.0,18601231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/CCCma/CanES...
1,CMIP,CCCma,CanESM5,historical,r10i1p1f1,day,ua,gn,NaN,v20190429,...,day,mean,Eastward Wind,m s-1,atmos,NaN,18610101.0,18701231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/CCCma/CanES...
2,CMIP,CCCma,CanESM5,historical,r10i1p1f1,day,ua,gn,NaN,v20190429,...,day,mean,Eastward Wind,m s-1,atmos,NaN,18710101.0,18801231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/CCCma/CanES...
3,CMIP,CCCma,CanESM5,historical,r10i1p1f1,day,ua,gn,NaN,v20190429,...,day,mean,Eastward Wind,m s-1,atmos,NaN,18810101.0,18901231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/CCCma/CanES...
4,CMIP,CCCma,CanESM5,historical,r10i1p1f1,day,ua,gn,NaN,v20190429,...,day,mean,Eastward Wind,m s-1,atmos,NaN,18910101.0,19001231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/CCCma/CanES...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,CMIP,MPI-M,MPI-ESM1-2-LR,historical,r9i1p1f1,day,ua,gn,NaN,v20190710,...,day,mean,Eastward Wind,m s-1,atmos,NaN,19300101.0,19491231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...
1050,CMIP,MPI-M,MPI-ESM1-2-LR,historical,r9i1p1f1,day,ua,gn,NaN,v20190710,...,day,mean,Eastward Wind,m s-1,atmos,NaN,19500101.0,19691231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...
1051,CMIP,MPI-M,MPI-ESM1-2-LR,historical,r9i1p1f1,day,ua,gn,NaN,v20190710,...,day,mean,Eastward Wind,m s-1,atmos,NaN,19700101.0,19891231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...
1052,CMIP,MPI-M,MPI-ESM1-2-LR,historical,r9i1p1f1,day,ua,gn,NaN,v20190710,...,day,mean,Eastward Wind,m s-1,atmos,NaN,19900101.0,20091231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...


In [6]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 365}},)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.source_id.experiment_id.table_id.grid_label'



KeyboardInterrupt



In [7]:
dset_dict = cat.to_dataset_dict()#cdf_kwargs={'chunks': {'time': 365}},)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.source_id.experiment_id.table_id.grid_label'


KeyboardInterrupt: 

# Selection and calculation

In [54]:
user = 'b381258'
outpath=f'/work/bd1022/{user}/'

for k, ds in dset_dict.items():
    ds_sel = ds['ua'].sel(
        plev = [1000],
        time = slice(None,'2100-12-31'),
        lat = 60,
        method = 'nearest'
    )
    ds_sel_winter = 
    output = f'{outpath}CMIP6/{k}_zg@10and100hPa.nc'
    print(output)
    
    lons = ds_sel.lon.values
    lats = ds_sel.lat.values
    xypoints = calc_xypoints(lons,lats)
    
    
    temp = ds_sel.sel(time = np.isin(ds_sel.time.dt.month, [11,12,1,2,3,4])).sel(lat = 60, method = 'nearest').mean(['lon','time'])#.load()



    sys.exit()

    
    #ds_sel.to_netcdf(output)
    

/mnt/lustre02/work/bd1022/b381258/CMIP6/ScenarioMIP.MPI-M.MPI-ESM1-2-LR.ssp585.day.gn_zg@10and100hPa.nc


<ipython-input-54-bd40e1a4cd1f>:19: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  moments_xr = xr.apply_ufunc(moments_fce, ds_sel, temp, \


SystemExit: 

/sw/spack-rhel6/miniforge3-4.9.2-3-Linux-x86_64-pwdbqi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
